# Hyper Parameter Tuning for classifier for four modes

In [1]:
import numpy as np
import pandas as pd
import os

# Classifier

In [9]:
import pandas as pd
import numpy as np
import os
# from os import path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report
from numpy import vstack, hstack, stack
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
import lightgbm
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

import pickle

In [3]:
# from Training_Code.config import SELECTCOLS, ECG_SELECTCOLS, EDA_SELECTCOLS, SELECTFOUR
# from sklearn.utils import shuffle

In [25]:
mainPath = r'X:\TAFFC_dataset\data'

subjectsList = os.listdir(mainPath)
subjectsList = [subjects.split('.')[0] for subjects in subjectsList]
dataPickle = {}
for subjects in subjectsList:
    dataPickle[subjects] = pd.read_csv(os.path.join(mainPath, f'{subjects}.csv'))


path_pickle = r'X:\TAFFC_dataset\pickle'

with open(os.path.join(path_pickle, 'data_clare.pickle'), 'wb') as handle:
    pickle.dump(dataPickle, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [26]:
mainPath = r'X:\TAFFC_dataset\pickle'

with open(os.path.join(mainPath, 'data_clare.pickle'), 'rb') as handle:
    dataPickle = pickle.load(handle)

In [27]:
for subjects in dataPickle.keys():
    scaler = StandardScaler()

    test_data = dataPickle[subjects]
    test_label = test_data['label'].values
    test_label = [1 if x > 5 else 0 for x in test_label]
    X_test = test_data.iloc[:, :-1]

    
    data = pd.DataFrame()

    for k, v in dataPickle.items():
        if k != subjects:
            data = pd.concat([data, v], axis = 0)

    X = data.iloc[:, :-1]
    train_label = data.iloc[:, -1]
    y = [1 if x < 5 else 0 for x in train_label]

    X = scaler.fit_transform(X)
    X_test = scaler.transform(X_test)

    random_grid = {'bootstrap': [True, False],
    'max_depth': [30, 50, 70, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 800], 'class_weight': ['balanced']}

    rf = RandomForestClassifier()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(X, y)
    
    break

    # data = [v for k, v in dataPickle.items() if k != subjects]

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [28]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'class_weight': 'balanced',
 'bootstrap': True}

In [29]:
rf_best = {'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'class_weight': 'balanced',
 'bootstrap': True}

In [31]:
for subjects in dataPickle.keys():
    scaler = StandardScaler()

    test_data = dataPickle[subjects]
    test_label = test_data['label'].values
    test_label = [1 if x > 5 else 0 for x in test_label]
    X_test = test_data.iloc[:, :-1]

    
    data = pd.DataFrame()

    for k, v in dataPickle.items():
        if k != subjects:
            data = pd.concat([data, v], axis = 0)

    X = data.iloc[:, :-1]
    train_label = data.iloc[:, -1]
    y = [1 if x < 5 else 0 for x in train_label]

    # X = scaler.fit_transform(X)
    # X_test = scaler.transform(X_test)

    model = RandomForestClassifier(**rf_best)
    hist = model.fit(X, y)
    predicted_y = model.predict(X_test)
    print(classification_report(test_label, predicted_y))

    # random_grid = {'bootstrap': [True, False],
    # 'max_depth': [30, 50, 70, None],
    # 'max_features': ['auto', 'sqrt'],
    # 'min_samples_leaf': [1, 2, 4],
    # 'min_samples_split': [2, 5, 10],
    # 'n_estimators': [100, 800], 'class_weight': ['balanced']}



              precision    recall  f1-score   support

           0       0.35      0.73      0.48       165
           1       0.19      0.04      0.07       231

    accuracy                           0.33       396
   macro avg       0.27      0.39      0.27       396
weighted avg       0.26      0.33      0.24       396

              precision    recall  f1-score   support

           0       0.05      0.52      0.09        33
           1       0.64      0.08      0.14       363

    accuracy                           0.12       396
   macro avg       0.35      0.30      0.12       396
weighted avg       0.59      0.12      0.14       396



c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.72      1.00      0.84       231
           1       0.00      0.00      0.00        88

    accuracy                           0.72       319
   macro avg       0.36      0.50      0.42       319
weighted avg       0.52      0.72      0.61       319

              precision    recall  f1-score   support

           0       0.20      0.91      0.32        77
           1       0.77      0.07      0.14       308

    accuracy                           0.24       385
   macro avg       0.48      0.49      0.23       385
weighted avg       0.65      0.24      0.17       385

              precision    recall  f1-score   support

           0       0.72      0.98      0.83       286
           1       0.00      0.00      0.00       110

    accuracy                           0.71       396
   macro avg       0.36      0.49      0.41       396
weighted avg       0.52      0.71      0.60       396

              preci

c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.38      1.00      0.55        33
           1       0.00      0.00      0.00        55

    accuracy                           0.38        88
   macro avg       0.19      0.50      0.27        88
weighted avg       0.14      0.38      0.20        88

              precision    recall  f1-score   support

           0       0.65      0.92      0.76       198
           1       0.06      0.01      0.02        99

    accuracy                           0.62       297
   macro avg       0.36      0.47      0.39       297
weighted avg       0.45      0.62      0.52       297

              precision    recall  f1-score   support

           0       0.04      0.55      0.07        11
           1       0.96      0.45      0.62       264

    accuracy                           0.46       275
   macro avg       0.50      0.50      0.35       275
weighted avg       0.92      0.46      0.60       275

              preci

c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anubhav\anaconda3\envs\cogl\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.93      0.96       297
           1       0.00      0.00      0.00         0

    accuracy                           0.93       297
   macro avg       0.50      0.46      0.48       297
weighted avg       1.00      0.93      0.96       297

              precision    recall  f1-score   support

           0       0.35      0.93      0.51       143
           1       0.44      0.03      0.06       253

    accuracy                           0.36       396
   macro avg       0.40      0.48      0.28       396
weighted avg       0.41      0.36      0.22       396

              precision    recall  f1-score   support

           0       0.98      0.56      0.71       385
           1       0.03      0.55      0.06        11

    accuracy                           0.56       396
   macro avg       0.51      0.55      0.39       396
weighted avg       0.95      0.56      0.70       396

              preci

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [17]:
def read_dataset(dataset, folder, basefolder):
    dr_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Features_Combined_scld
    bs_feat_path = r'X:\All Modes\All\{}\Combine'.format(dataset) # ECG_EDA_Base2_Features_Combined

    XtrainDriv = make_kfolds(dr_feat_path)
    # XtrainBase = make_kfolds(bs_feat_path)

    XtrainDriv = XtrainDriv[SELECTFOUR].copy()
    ytrainDriv = list(XtrainDriv['scaled label'].copy())

    # XtrainBase = XtrainBase[SELECTFOUR[:-3]].copy()
    # ytrainBase = XtrainBase.shape[0] * [0]

    XtrainDriv.drop(columns=['label', 'complexity', 'scaled label'], inplace=True)
    # XtrainDriv = XtrainDriv.append(XtrainBase)

    ytrain = ytrainDriv # + ytrainBase

    X = XtrainDriv.values
    
    X, ytrain = shuffle(X, ytrain, random_state=42)

    for idx, val in enumerate(ytrain):
        if val <= 4:
            ytrain[idx] = 0
        else: ytrain[idx] = 1

    return X, ytrain

## Hyperparameter tuning

In [84]:
data, label = read_dataset('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

In [85]:
X_train.shape

(2352, 315)

In [86]:
random_grid = {'bootstrap': [True, False],
 'max_depth': [30, 50, 70, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 800, 1000, 2000, 3000], 'class_weight': ['balanced']}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced'],
                                        'max_depth': [30, 50, 70, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 800, 1000, 2000,
                                                         3000]},
                   random_state=42, verbose=2)

In [87]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'class_weight': 'balanced',
 'bootstrap': False}

In [ ]:
{'n_estimators': 2000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': False}

In [88]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.74      0.68      0.71       260
           1       0.76      0.81      0.78       329

    accuracy                           0.75       589
   macro avg       0.75      0.75      0.75       589
weighted avg       0.75      0.75      0.75       589



In [20]:
data, label = read_dataset('Virage', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

In [21]:
X_train.shape

(2141, 315)

In [22]:
random_grid = {'bootstrap': [True, False],
 'max_depth': [30, 50, 70, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 800, 1000, 2000, 3000]}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [30, 50, 70, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 800, 1000, 2000,
                                                         3000]},
                   random_state=42, verbose=2)

In [23]:
rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': False}

In [24]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.87      0.69      0.77       205
           1       0.83      0.94      0.88       331

    accuracy                           0.84       536
   macro avg       0.85      0.81      0.82       536
weighted avg       0.84      0.84      0.84       536



In [25]:
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(model)
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=False, max_depth=30,
                                                 n_estimators=2000))

# SVM

In [60]:
data, label = read_dataset('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

parameters = {'C': [1, 5, 15, 20], 'kernel':['rbf'], 'class_weight': ['balanced']}

svc_ = SVC()
rf_random = RandomizedSearchCV(estimator = svc_, param_distributions = parameters, n_iter = 50, cv = 10, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

C:\Users\Anubhav\anaconda3\envs\ideas\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 10 folds for each of 4 candidates, totalling 40 fits


RandomizedSearchCV(cv=10, estimator=SVC(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': [1, 5, 15, 20],
                                        'class_weight': ['balanced'],
                                        'kernel': ['rbf']},
                   random_state=42, verbose=2)

In [61]:
X_train.shape

(2352, 315)

In [62]:
rf_random.best_params_

{'kernel': 'rbf', 'class_weight': 'balanced', 'C': 5}

In [63]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.69      0.73      0.71       260
           1       0.78      0.74      0.76       329

    accuracy                           0.74       589
   macro avg       0.74      0.74      0.74       589
weighted avg       0.74      0.74      0.74       589



In [57]:
data, label = read_dataset('Virage', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

parameters = {'C': [1, 5, 15, 20], 'kernel':['rbf'], 'class_weight': ['balanced']}

svc_ = SVC()
rf_random = RandomizedSearchCV(estimator = svc_, param_distributions = parameters, n_iter = 50, cv = 10, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

C:\Users\Anubhav\anaconda3\envs\ideas\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 10 folds for each of 4 candidates, totalling 40 fits


RandomizedSearchCV(cv=10, estimator=SVC(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': [1, 5, 15, 20],
                                        'class_weight': ['balanced'],
                                        'kernel': ['rbf']},
                   random_state=42, verbose=2)

In [58]:
rf_random.best_params_

{'kernel': 'rbf', 'class_weight': 'balanced', 'C': 20}

In [59]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76       205
           1       0.84      0.88      0.86       331

    accuracy                           0.82       536
   macro avg       0.82      0.81      0.81       536
weighted avg       0.82      0.82      0.82       536



# Logistic Regression

In [75]:
from sklearn.preprocessing import StandardScaler

data, label = read_dataset('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')

scaler = StandardScaler()
data = scaler.fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)


parameters_lr = {'C': [0.5, 1, 5, 10], 'max_iter': [400, 500, 600, 1000], 'class_weight': ['balanced']}

lr_ = LogisticRegression()
rf_random = RandomizedSearchCV(estimator = lr_, param_distributions = parameters_lr, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

C:\Users\Anubhav\anaconda3\envs\ideas\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 16 is smaller than n_iter=100. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits


RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [0.5, 1, 5, 10],
                                        'class_weight': ['balanced'],
                                        'max_iter': [400, 500, 600, 1000]},
                   random_state=42, verbose=2)

In [76]:
rf_random.best_params_

{'max_iter': 400, 'class_weight': 'balanced', 'C': 0.5}

In [77]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63       260
           1       0.71      0.67      0.69       329

    accuracy                           0.66       589
   macro avg       0.66      0.66      0.66       589
weighted avg       0.67      0.66      0.66       589



In [72]:
from sklearn.preprocessing import StandardScaler

data, label = read_dataset('Virage', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')

scaler = StandardScaler()
data = scaler.fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)


parameters_lr = {'C': [0.5, 1, 5, 10], 'max_iter': [1000, 1500], 'class_weight': ['balanced']}

lr_ = LogisticRegression()
rf_random = RandomizedSearchCV(estimator = lr_, param_distributions = parameters_lr, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

C:\Users\Anubhav\anaconda3\envs\ideas\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 8 is smaller than n_iter=100. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits


RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [0.5, 1, 5, 10],
                                        'class_weight': ['balanced'],
                                        'max_iter': [1000, 1500]},
                   random_state=42, verbose=2)

In [73]:
rf_random.best_params_

{'max_iter': 1000, 'class_weight': 'balanced', 'C': 0.5}

In [74]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.65      0.79      0.71       205
           1       0.85      0.74      0.79       331

    accuracy                           0.76       536
   macro avg       0.75      0.76      0.75       536
weighted avg       0.77      0.76      0.76       536



# LGBM

In [78]:
data, label = read_dataset('MatbII', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

import lightgbm as lgb
lgbmcls = lgb.LGBMClassifier()
default_params = lgbmcls.get_params()

params = {'boosting_type': ['gbdt', 'dart'],
 'class_weight': ['balanced'],
 'importance_type': ['split', 'gains'],
 'learning_rate': [0.1, 0.01, 0.05],
 'n_estimators': [1000, 500, 200, 1500],
 'num_leaves': [100, 500, 1000],
 'reg_lambda': [0.01, 0.001]}

lgb_ = lgb.LGBMClassifier()
rf_random = RandomizedSearchCV(estimator = lgb_, param_distributions = params, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'boosting_type': ['gbdt', 'dart'],
                                        'class_weight': ['balanced'],
                                        'importance_type': ['split', 'gains'],
                                        'learning_rate': [0.1, 0.01, 0.05],
                                        'n_estimators': [1000, 500, 200, 1500],
                                        'num_leaves': [100, 500, 1000],
                                        'reg_lambda': [0.01, 0.001]},
                   random_state=42, verbose=2)

In [79]:
rf_random.best_params_

{'reg_lambda': 0.01,
 'num_leaves': 100,
 'n_estimators': 1000,
 'learning_rate': 0.1,
 'importance_type': 'gains',
 'class_weight': 'balanced',
 'boosting_type': 'dart'}

In [80]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71       260
           1       0.76      0.83      0.80       329

    accuracy                           0.76       589
   macro avg       0.76      0.75      0.76       589
weighted avg       0.76      0.76      0.76       589



In [81]:
data, label = read_dataset('Virage', 'ECG_EDA_Features_Combined_scld', 'ECG_EDA_Base2_Features_Combined')
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42, shuffle=True)

import lightgbm as lgb
lgbmcls = lgb.LGBMClassifier()
default_params = lgbmcls.get_params()

params = {'boosting_type': ['gbdt', 'dart'],
 'class_weight': ['balanced'],
 'importance_type': ['split', 'gains'],
 'learning_rate': [0.1, 0.01, 0.05],
 'n_estimators': [1000, 500, 200, 1500],
 'num_leaves': [100, 500, 1000],
 'reg_lambda': [0.01, 0.001]}

lgb_ = lgb.LGBMClassifier()
rf_random = RandomizedSearchCV(estimator = lgb_, param_distributions = params, n_iter = 25, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=25, n_jobs=-1,
                   param_distributions={'boosting_type': ['gbdt', 'dart'],
                                        'class_weight': ['balanced'],
                                        'importance_type': ['split', 'gains'],
                                        'learning_rate': [0.1, 0.01, 0.05],
                                        'n_estimators': [1000, 500, 200, 1500],
                                        'num_leaves': [100, 500, 1000],
                                        'reg_lambda': [0.01, 0.001]},
                   random_state=42, verbose=2)

In [82]:
rf_random.best_params_

{'reg_lambda': 0.01,
 'num_leaves': 100,
 'n_estimators': 1000,
 'learning_rate': 0.1,
 'importance_type': 'gains',
 'class_weight': 'balanced',
 'boosting_type': 'dart'}

In [83]:
model = rf_random.best_estimator_
hist = model.fit(X_train, y_train)
predicted_y = model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.84      0.76      0.80       205
           1       0.86      0.91      0.89       331

    accuracy                           0.85       536
   macro avg       0.85      0.84      0.84       536
weighted avg       0.85      0.85      0.85       536

